In [1]:
import accelerate
import transformers
import torch

print(f"Accelerate sürümü: {accelerate.__version__}")
print(f"Transformers sürümü: {transformers.__version__}")
print(f"PyTorch sürümü: {torch.__version__}")

Accelerate sürümü: 1.2.1
Transformers sürümü: 4.47.1
PyTorch sürümü: 2.5.1


1. Tokenizasyon

In [2]:
import pandas as pd

# Veri setini yükleme
bert_data = pd.read_csv("data/bert_prepared_data.csv")

# Eksik veya hatalı verileri kontrol et
bert_data['log_text'] = bert_data['log_text'].astype(str).fillna('')

# Tokenizasyonu tekrar deneyelim
from transformers import BertTokenizer

# BERT tokenizer'ı yükleyelim
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Tokenize etme
bert_data['tokenized'] = bert_data['log_text'].apply(
    lambda x: tokenizer(x, padding='max_length', truncation=True, return_tensors="pt")
)

# İlk tokenize edilmiş veriyi kontrol edelim
print(bert_data['tokenized'].iloc[0])

{'input_ids': tensor([[  101, 16798,  2629,  1011,  5890,  1011,  5890,  5840,  1024,  2321,
          1024,  2538,  1009,  6021,  1024,  4002,  1015,  1012,  6036,  1012,
         16065,  1012, 16333,  2131,  1013,  6904,  7903,  2239,  1012, 24582,
          2080, 28203,  8732,  2487,  8299,  1024,  1013,  1013,  7151, 15750,
          2099,  1012,  4012,  1013,  9587,  5831,  4571,  1013,  1019,  1012,
          1014,  1006, 22228,  1025, 13420,  6097,  9808,  1060,  2184,  1035,
          2321,  1035,  1021,  1007,  6207,  8545,  2497, 23615,  1013,  5187,
          2581,  1012,  4029,  1006,  1047, 11039, 19968,  1010,  2066, 16216,
         19665,  1007, 18546,  1013,  6205,  1012,  1014,  1012,  4008,  2581,
          2475,  1012, 13412, 23591,  1013,  5187,  2581,  1012,  4029,   102,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,  

2. Eğitim ve Test verilerinin ayrımı

In [3]:
from sklearn.model_selection import train_test_split

# Tokenize edilmiş veriyi ve etiketleri ayır
X = bert_data['tokenized']
y = bert_data['label']

# Veriyi eğitim ve test setine ayır
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Eğitim verisi boyutu: {len(X_train)}")
print(f"Test verisi boyutu: {len(X_test)}")

Eğitim verisi boyutu: 8000
Test verisi boyutu: 2000


 3. BERT modeli ile eğitim için veri hazırlığı

In [4]:
import torch

# Tokenize edilmiş verileri PyTorch tensorlerine dönüştür
train_encodings = torch.stack([item['input_ids'].squeeze(0) for item in X_train])
train_labels = torch.tensor(y_train.values)

test_encodings = torch.stack([item['input_ids'].squeeze(0) for item in X_test])
test_labels = torch.tensor(y_test.values)

# PyTorch Dataset oluştur
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            'input_ids': self.encodings[idx],
            'labels': self.labels[idx]
        }

train_dataset = CustomDataset(train_encodings, train_labels)
test_dataset = CustomDataset(test_encodings, test_labels)

4. BERT modeli yükleme

In [5]:
from transformers import BertForSequenceClassification

# BERT modelini yükleyelim (Binary Classification için 2 sınıf)
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


5.  Eğitim Parametreleri ve Eğitici (Trainer) Tanımlama

In [7]:
from transformers import Trainer, TrainingArguments

# Eğitim parametrelerini tanımla
training_args = TrainingArguments(
    output_dir='./results',         # Modelin kaydedileceği dizin
    num_train_epochs=3,             # Eğitim epoch sayısı
    per_device_train_batch_size=16, # Eğitim sırasında her cihaz için batch size
    per_device_eval_batch_size=64,  # Değerlendirme sırasında batch size
    warmup_steps=500,               # Isınma adımları
    weight_decay=0.01,              # Ağırlık zayıflatma
    logging_dir='./logs',           # Logların kaydedileceği dizin
    logging_steps=10,               # Her 10 adımdan bir log al
    eval_strategy ="epoch",         # Her epoch'ta değerlendirme yap
)

# Eğitici (Trainer) tanımla
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

In [ ]:
trainer.train()

  0%|          | 0/1500 [00:00<?, ?it/s]

We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


{'loss': 0.7091, 'grad_norm': 9.142970085144043, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.02}
